# Sample Projects
This notebook is intended as a playground to try out various python widgets, ideas etc. This notebook is publically saved on github.

In [1]:
# tax rate is fixed
TAX_RATE = .0605

In [28]:
from math import floor

def round(x):
    return int(((x * 1000) + 5)/10)/100.0

class LineItem:
    def __init__(self,label,each_amount,qty):
        self.label = label
        self.each_amount = each_amount
        self.qty = qty
        self.line_amount = round(self.each_amount * self.qty)
    def __str__(self):
        if self.label == "LTAX":
            return "{} {} x {:0,.4f} = ${:0,.2f}".format(self.label,self.qty,self.each_amount,self.line_amount)
        return "{} {} x ${:0,.2f} = ${:0,.2f}".format(self.label,self.qty,self.each_amount,self.line_amount)

def invoice_str(invoice):
    return [str(i) for i in invoice]

In [3]:
def simple_prediction(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,npets,extra_fee,rate):
    predicted_nightly_rate = round((gross_paid - tax_paid - cln_fee - extra_fee - (pet_fee * npets)) / stay_nights)
    rent = (predicted_nightly_rate * stay_nights)
    pets = pet_fee * npets
    subtotal = rent + cln_fee + pets + extra_fee
    tax_calc = round(subtotal * rate)
    total = subtotal + tax_calc
    invoice = [
        LineItem("RNT",predicted_nightly_rate,stay_nights),
        LineItem("CLN",cln_fee,1),
        LineItem("PET",pet_fee,npets),
        LineItem("XFEE",extra_fee,1),
        LineItem("LTAX",rate,subtotal)]
    print("Trying invoice={}".format(invoice_str(invoice)))
    
    if total != gross_paid:
        print("Invoice FAILED!!")
        raise RuntimeError("Could not predict invoice!")
        
    print("Invoice succeeded.")
    return invoice

In [34]:
def predict_invoice(stay_nights,gross_paid,tax_paid,cln_fee=135.00,pet_fee=95.00,extra_fee=0.0, rate=.0605):
    """
    Predicts the invoice line items given stay nights, gross amount paid,
    taxes paid, cleaning fee, and pet fees.  If the invoice can not be predicted,
    raise an exception.
    
    Parameters
    ----------
      stay_nights : integer
          The number of nights stayed. Used to multiple the nightly rate to determine overal rent.
      gross_paid : double
          The amount (USD) paid for the stay
      tax_paid : double
          The amount (USD) paid in taxes.  This should be total amount (rent + fees) * TAX_RATE.
      cln_fee : double
          The amount (USD) charged/paid for cleaning fees.
      pet_fee : double
          The amount (USD) charged/paid for pet fees.
    
    Returns
    -------
       A list of invoice items or raises an exception if the invoice could not be created
    """
    
#    # initialize the list of predictions
#    # a prediction is just a function that tries to return a list of invoice items
#    predictions = [simple_prediction]

    print("Predicting invoice:")
    print("   (stay_nights :: gross_paid :: tax_paid) = ({} :: ${:0,.2f} :: ${:0,.2f})".format(stay_nights,gross_paid,tax_paid))
#    prediction = predictions.pop()
    invoice = []
    npets = 0
    while True:
        try:
#            invoice = prediction(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,npets)
            invoice = simple_prediction(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,npets,extra_fee,rate)
            break
        except RuntimeError:
#            try:
#                prediction = predictions.pop()
#            except IndexError:
#                raise RuntimeError("Could not predict invoice!")
            npets += 1
            if npets==3 and rate==.05:
                raise RuntimeError("Could not predict invoice!")
            elif npets==3:
                return predict_invoice(stay_nights,gross_paid,tax_paid,cln_fee,pet_fee,extra_fee,rate=.05)

    return invoice

In [41]:
import csv

with open('payments.tsv') as payments_file:
    values_tsv = csv.reader(payments_file, delimiter='\t')
    data=[row for row in values_tsv]
    data=data[1:]
data

[['HA-WB4320', '20210407', '4', '1346.84', '43.08', '1303.76', '76.84', '0'],
 ['HA-1HMTV0', '20210409', '4', '1055.25', '33.76', '1021.49', '50.25', '0'],
 ['HA-3CGXD6', '20210411', '4', '1182.3', '37.82', '1144.48', '56.3', '0'],
 ['HA-FHTQ5D', '20210415', '6', '1527.75', '48.88', '1478.87', '72.75', '0'],
 ['HA-FRQ7Y2', '20210419', '4', '1065.75', '34.09', '1031.66', '50.75', '0'],
 ['HA-6JQCXB', '20210421', '5', '1396.5', '44.68', '1351.82', '66.5', '0'],
 ['HA-FZBJCY', '20210422', '3', '763.56', '24.57', '738.99', '43.56', '0'],
 ['HA-4KC69Y', '20210424', '3', '988.39', '31.62', '956.77', '56.39', '0'],
 ['HA-FWCCZR', '20210426', '4', '1065.75', '34.09', '1031.66', '50.75', '0'],
 ['HA-281H9J', '20210501', '7', '1696.8', '54.59', '1642.21', '80.8', '0'],
 ['HA-240K0B', '20210501', '5', '1262', '40.61', '1221.39', '72', '0']]

In [42]:
def invoice_label(row):
    return "{} co {}:{}".format(row[0],row[1],row[3])

def to_invoice(row):
    try:
        return invoice_str(predict_invoice(stay_nights=int(row[2]),
                                          gross_paid=float(row[3]),
                                          tax_paid=float(row[6]),
                                          extra_fee=float(row[7])))
    except Exception as e:
        print(e)
        return "INVOICE FAIL"

results = {invoice_label(row):to_invoice(row) for row in data}
results

Predicting invoice:
   (stay_nights :: gross_paid :: tax_paid) = (4 :: $1,346.84 :: $76.84)
Trying invoice=['RNT 4 x $283.75 = $1,135.00', 'CLN 1 x $135.00 = $135.00', 'PET 0 x $95.00 = $0.00', 'XFEE 1 x $0.00 = $0.00', 'LTAX 1270.0 x 0.0605 = $76.84']
Invoice succeeded.
Predicting invoice:
   (stay_nights :: gross_paid :: tax_paid) = (4 :: $1,055.25 :: $50.25)
Trying invoice=['RNT 4 x $217.50 = $870.00', 'CLN 1 x $135.00 = $135.00', 'PET 0 x $95.00 = $0.00', 'XFEE 1 x $0.00 = $0.00', 'LTAX 1005.0 x 0.0605 = $60.80']
Invoice FAILED!!
Trying invoice=['RNT 4 x $193.75 = $775.00', 'CLN 1 x $135.00 = $135.00', 'PET 1 x $95.00 = $95.00', 'XFEE 1 x $0.00 = $0.00', 'LTAX 1005.0 x 0.0605 = $60.80']
Invoice FAILED!!
Trying invoice=['RNT 4 x $170.00 = $680.00', 'CLN 1 x $135.00 = $135.00', 'PET 2 x $95.00 = $190.00', 'XFEE 1 x $0.00 = $0.00', 'LTAX 1005.0 x 0.0605 = $60.80']
Invoice FAILED!!
Predicting invoice:
   (stay_nights :: gross_paid :: tax_paid) = (4 :: $1,055.25 :: $50.25)
Trying invoic

{'HA-WB4320 co 20210407:1346.84': ['RNT 4 x $283.75 = $1,135.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $0.00 = $0.00',
  'LTAX 1270.0 x 0.0605 = $76.84'],
 'HA-1HMTV0 co 20210409:1055.25': ['RNT 4 x $217.50 = $870.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $0.00 = $0.00',
  'LTAX 1005.0 x 0.0500 = $50.25'],
 'HA-3CGXD6 co 20210411:1182.3': ['RNT 4 x $247.75 = $991.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $0.00 = $0.00',
  'LTAX 1126.0 x 0.0500 = $56.30'],
 'HA-FHTQ5D co 20210415:1527.75': ['RNT 6 x $220.00 = $1,320.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $0.00 = $0.00',
  'LTAX 1455.0 x 0.0500 = $72.75'],
 'HA-FRQ7Y2 co 20210419:1065.75': ['RNT 4 x $220.00 = $880.00',
  'CLN 1 x $135.00 = $135.00',
  'PET 0 x $95.00 = $0.00',
  'XFEE 1 x $0.00 = $0.00',
  'LTAX 1015.0 x 0.0500 = $50.75'],
 'HA-6JQCXB co 20210421:1396.5': ['RNT 5 x $239.00 = $1,195.00